<a href="https://colab.research.google.com/github/shcho11/00.Projects_KoreaUniv_2022-2/blob/main/202212_RME_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
!pip install text_utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
!pip install python-text-utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [50]:
!pip install constants

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for constants: filename=constants-0.6.0-py3-none-any.whl size=5473 sha256=14ba023dc6dad6f92a8b949fc0c9215b2440ef478b27b2575e8c78ad4c0510a6
  Stored in directory: /root/.cache/pip/wheels/77/ac/b2/89268490b92bf6fd0102b3634668042437e0e024c64ef447a1
Successfully built constants


In [58]:
!pip install wmf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 4.2 MB/s 
  Created wheel for wmf: filename=wmf-0.4.2-cp38-cp38-linux_x86_64.whl size=585614 sha256=7724902140f8adb91e395167a541b5a79727fabc8f8df70e81765f3d9235838d
  Stored in directory: /root/.cache/pip/wheels/f1/39/3d/8eceaf95902c25ab77105ef03a69fb755af6192b682f26a2f0
Successfully built wmf


In [61]:
!pip install pyll

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 5.1 MB/s 
  Using cached Paste-3.5.2-py2.py3-none-any.whl (593 kB)
  Using cached PasteScript-3.2.1-py2.py3-none-any.whl (73 kB)
     |████████████████████████████████| 66 kB 5.8 MB/s 
     |████████████████████████████████| 363 kB 59.1 MB/s 
     |████████████████████████████████| 177 kB 68.1 MB/s 
     |████████████████████████████████| 85 kB 5.0 MB/s 
     |████████████████████████████████| 729 kB 49.6 MB/s 
     |████████████████████████████████| 47 kB 4.5 MB/s 
     |████████████████████████████████| 114 kB 60.0 MB/s 
  Using cached PasteDeploy-3.0.1-py3-none-any.whl (16 kB)
     |████████████████████████████████| 261 kB 29.3 MB/s 
  Created wheel for pyll: filename=Pyll-0.4.3-py3-none-any.whl size=1188720 sha256=f3929b9436aa87a1f72c0b90aa46e33d443b731785a2773202985995f537324a
  Stored in directory: /root/.cache/pip/wheels/7f/da/d1/e7caf

In [65]:
!apt install produce_negative_embedding

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package produce_negative_embedding


In [54]:
import itertools
import glob
import os
import sys
os.environ['OPENBLAS_NUM_THREADS'] = '1'
import numpy as np
import time
import python_text_utils
import pandas as pd
from scipy import sparse
from sklearn.utils import shuffle
from joblib import Parallel, delayed
import constants as gc

np.random.seed(98765) #set random seed

import argparse

In [91]:
parser = argparse.ArgumentParser("Description: Running multi-embedding recommendation - RME model")
parser.add_argument('-f')
parser.add_argument('--data_path', default='data', type=str, help='path to the data')
parser.add_argument('--batch_size', default=5000, type=float, help='batch processing')
parser.add_argument('--dataset', default="ml10m", type=str, help='dataset')
args = parser.parse_args()


In [10]:
os.getcwd()

'/content'

In [17]:
DATA_DIR =  os.path.join('drive', 'MyDrive', 'dl2022')

unique_uid = list()
with open(os.path.join(DATA_DIR, 'unique_uid.txt'), 'r') as f:
    for line in f:
        unique_uid.append(line.strip())

unique_movieId = list()
with open(os.path.join(DATA_DIR, 'unique_sid.txt'), 'r') as f:
    for line in f:
        unique_movieId.append(line.strip())
n_items = len(unique_movieId)
n_users = len(unique_uid)

print('number of users:%d ,  number of items: %d'%(n_users, n_items))

number of users:20279 ,  number of items: 6051


In [18]:
def load_data(csv_file, shape=(n_users, n_items)):
    tp = pd.read_csv(csv_file)
    rows, cols = np.array(tp['userId']), np.array(tp['movieId']) #rows will be user ids, cols will be item-ids.
    seq = np.concatenate((  rows[:, None], cols[:, None], np.ones((rows.size, 1), dtype='int')
                          ), axis=1)
    data = sparse.csr_matrix((np.ones_like(rows), (rows, cols)), dtype=np.int16, shape=shape)
    return data, seq, tp

In [26]:
####################Generate item-item co-occurrence matrix based on the user's consumed items history ############
# ##################       This will build a item-item co-occurrence matrix           ############################
#user 1: item 1, item 2, ... item k --> item 1, 2, ..., k will be seen as a sentence ==> do co-occurrence.

def _coord_batch(lo, hi, train_data, prefix = 'item', max_neighbor_words = 100000, choose='macro'):
    rows = []
    cols = []

    for u in range(lo, hi):
        #print train_data[u].nonzero()[1] #names all the item ids that the user at index u watched nonzero return a
        # 2D array, index 0 will be the row index and index 1 will be columns whose values are not equal to 0
        lst_words = train_data[u].nonzero()[1]
        if len(lst_words) > max_neighbor_words:
            if choose == 'micro':
                #approach 1: randomly select max_neighbor_words for each word.
                for w in lst_words:
                    tmp = lst_words.remove(w)
                    #random choose max_neigbor words in the list:
                    neighbors = np.random.choice(tmp, max_neighbor_words, replace=False)
                    for c in neighbors:
                        rows.append(w)
                        cols.append(c)
            if choose == 'macro':
                #approach 2: randomly select the sentence with length of max_neigbor_words + 1, then do permutation.
                lst_words = np.random.choice(lst_words, max_neighbor_words + 1, replace=False)
                for w, c in itertools.permutations(lst_words, 2):
                    rows.append(w)
                    cols.append(c)
        else:
            for w, c in itertools.permutations(lst_words, 2):
                rows.append(w)
                cols.append(c)
    if not os.path.exists(os.path.join(DATA_DIR, 'co-temp')): os.mkdir(os.path.join(DATA_DIR, 'co-temp'))
    np.save(os.path.join(DATA_DIR, 'co-temp' ,'%s_coo_%d_%d.npy' % (prefix, lo, hi)),
            np.concatenate([np.array(rows)[:, None], np.array(cols)[:, None]], axis=1)) #append column wise.
    pass


batch_size = args.batch_size

train_data, train_raw, train_df = load_data(os.path.join(DATA_DIR, 'train.csv'))
#clear the co-temp folder:
if os.path.exists(os.path.join(DATA_DIR, 'co-temp')):
    for f in glob.glob(os.path.join(DATA_DIR, 'co-temp', '*.npy')):
        os.remove(f)

GENERATE_ITEM_ITEM_COOCCURENCE_FILE = True
if GENERATE_ITEM_ITEM_COOCCURENCE_FILE:
    t1 = time.time()
    print ('Generating item item co-occurrence matrix')
    start_idx = range(0, n_users, batch_size)
    list_start_idx = list(start_idx)
    end_idx = list_start_idx[1:] + [n_users]
    Parallel(n_jobs=1)(delayed(_coord_batch)(lo, hi, train_data, prefix = 'item', max_neighbor_words = 200) for lo, hi in zip(start_idx, end_idx))
    t2 = time.time()
    print ('Time : %d seconds'%(t2-t1))
    pass

Generating item item co-occurrence matrix
Time : 48 seconds


In [81]:
import pickle

# save
def save_pickle(data, path):
    with open(path, 'wb') as f:
        pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
        print ('Saved %s..' %path)

# load
def load_pickle(path):
   with open(path, 'rb') as f:
      pickle.load(f)

In [82]:
########################################################################################################################
####################Generate user-user co-occurrence matrix based on the same items they backed######################
#####################        This will build a user-user co-occurrence matrix ##########################################

GENERATE_USER_USER_COOCCURENCE_FILE = True
if GENERATE_USER_USER_COOCCURENCE_FILE:
    t1 = time.time()
    print('Generating user user co-occurrence matrix')
    start_idx = range(0, n_items, batch_size)
    list_start_idx = list(start_idx)
    end_idx = list_start_idx[1:] + [n_items]
    Parallel(n_jobs=8)(delayed(_coord_batch)(lo, hi, train_data.T, prefix = 'user', max_neighbor_words=100) for lo, hi in zip(start_idx, end_idx))
    t2 = time.time()
    print('Time : %d seconds'%(t2 - t1))
    pass
########################################################################################################################

def _load_coord_matrix(start_idx, end_idx, nrow, ncol, prefix = 'item'):
    X = sparse.csr_matrix((nrow, ncol), dtype='float32')

    for lo, hi in zip(start_idx, end_idx):
        coords = np.load(os.path.join(DATA_DIR, 'co-temp', '%s_coo_%d_%d.npy' % (prefix, lo, hi)))

        rows = coords[:, 0]
        cols = coords[:, 1]

        tmp = sparse.coo_matrix((np.ones_like(rows), (rows, cols)), shape=(nrow, ncol), dtype='float32').tocsr()
        X = X + tmp

        print("%s %d to %d finished" % (prefix, lo, hi))
        sys.stdout.flush()
    return X

BOOLEAN_LOAD_PP_COOCC_FROM_FILE = True
X, Y = None, None
if BOOLEAN_LOAD_PP_COOCC_FROM_FILE:
    print ('Loading item item co-occurrence matrix')
    t1 = time.time()
    start_idx = range(0, n_users, batch_size)
    list_start_idx = list(start_idx)
    end_idx = list_start_idx[1:] + [n_users]
    X = _load_coord_matrix(start_idx, end_idx, n_items, n_items, prefix = 'item') #item item co-occurrence matrix
    print ('dumping matrix ...')
    save_pickle(X, os.path.join(DATA_DIR,'item_item_cooc.dat'))
    t2 = time.time()
    print ('Time : %d seconds'%(t2-t1))
else:
    print ('test loading model from pickle file')
    t1 = time.time()
    X = load_pickle(os.path.join(DATA_DIR,'item_item_cooc.dat'))
    t2 = time.time()
    print ('[INFO]: sparse matrix size of item-item co-occurrence matrix: %d mb\n' % (
                                                    (X.data.nbytes + X.indices.nbytes + X.indptr.nbytes) / (1024 * 1024)))
    print ('Time : %d seconds'%(t2-t1))

#X = None
BOOLEAN_LOAD_UU_COOCC_FROM_FILE = True
if BOOLEAN_LOAD_UU_COOCC_FROM_FILE:
    print ('Loading user user co-occurrence matrix')
    t1 = time.time()
    start_idx = range(0, n_items, batch_size)
    list_start_idx = list(start_idx)
    end_idx = list_start_idx[1:] + [n_items]
    Y = _load_coord_matrix(start_idx, end_idx, n_users, n_users, prefix = 'user') #user user co-occurrence matrix

    t2 = time.time()
    print ('Time : %d seconds' % (t2 - t1))

    print ('dumping matrix ...')
    t1 = time.time()
    save_pickle(Y, os.path.join(DATA_DIR, 'user_user_cooc.dat'))
    t2 = time.time()
    print ('Time : %d seconds'%(t2-t1))
else:
    print ('test loading model from pickle file')
    t1 = time.time()
    Y = load_pickle(os.path.join(DATA_DIR, 'user_user_cooc.dat'))
    t2 = time.time()
    print ('[INFO]: sparse matrix size of user user co-occurrence matrix: %d mb\n' % (
                                                    (Y.data.nbytes + Y.indices.nbytes + Y.indptr.nbytes) / (1024 * 1024)))
    print ('Time : %d seconds'%(t2-t1))

Generating user user co-occurrence matrix
Time : 33 seconds
Loading item item co-occurrence matrix
item 0 to 5000 finished
item 5000 to 10000 finished
item 10000 to 15000 finished
item 15000 to 20000 finished
item 20000 to 20279 finished
dumping matrix ...
Saved drive/MyDrive/dl2022/item_item_cooc.dat..
Time : 12 seconds
Loading user user co-occurrence matrix
user 0 to 5000 finished
user 5000 to 6051 finished
Time : 2 seconds
dumping matrix ...
Saved drive/MyDrive/dl2022/user_user_cooc.dat..
Time : 0 seconds


In [83]:
####################Generate item-item co-occurrence matrix based on the user backed items history ############
# ##################       This will build a item-item co-occurrence matrix           ############################
#user 1: item 1, item 2, ... item k --> item 1, 2, ..., k will be seen as a sentence ==> do co-occurrence.

def _coord_batch_n(lo, hi, train_data, prefix = 'item', max_neighbor_words = 200, choose='macro'):
    rows = []
    cols = []

    for u in range(lo, hi):
        #print train_data[u].nonzero()[1] #names all the item ids that the user at index u watched nonzero return a
        # 2D array, index 0 will be the row index and index 1 will be columns whose values are not equal to 0
        lst_words = train_data[u].nonzero()[1]
        #some users may have some hundreds of consumed items
        # --> It's terrible time consuming when perfoming permutations of this long sequence. --> limit max_neighbor_words
        # --> need to randomly pickup max_neighbor_words when the total of consumed items is > max_neighbor_words.
        if len(lst_words) > max_neighbor_words:
            if choose == 'micro':
                #approach 1: randomly select max_neighbor_words for each word.
                for w in lst_words:
                    tmp = lst_words.remove(w)
                    #random choose max_neigbor words in the list:
                    neighbors = np.random.choice(tmp, max_neighbor_words, replace=False)
                    for c in neighbors:
                        rows.append(w)
                        cols.append(c)
            if choose == 'macro':
                #approach 2: randomly select the sentence with length of max_neigbor_words + 1, then do permutation.
                lst_words = np.random.choice(lst_words, max_neighbor_words + 1, replace=False)
                for w, c in itertools.permutations(lst_words, 2):
                    rows.append(w)
                    cols.append(c)
        else:

            for w, c in itertools.permutations(lst_words, 2):
                rows.append(w)
                cols.append(c)
    if not os.path.exists(os.path.join(DATA_DIR, 'negative-co-temp')): os.mkdir(os.path.join(DATA_DIR, 'negative-co-temp'))
    np.save(os.path.join(DATA_DIR, 'negative-co-temp' ,'negative_%s_coo_%d_%d.npy' % (prefix, lo, hi)),
            np.concatenate([np.array(rows)[:, None], np.array(cols)[:, None]], axis=1)) #append column wise.
    pass


batch_size = args.batch_size


train_data, train_raw, train_df = load_data(os.path.join(DATA_DIR, 'train_neg.csv'))
#clear the negative-co-temp folder:
if os.path.exists(os.path.join(DATA_DIR, 'negative-co-temp')):
    for f in glob.glob(os.path.join(DATA_DIR, 'negative-co-temp', '*.npy')):
        os.remove(f)


t1 = time.time()
print ('Generating item item negative_co-occurrence matrix')
start_idx = range(0, n_users, batch_size)
list_start_idx = list(start_idx)
end_idx = list_start_idx[1:] + [n_users]
Parallel(n_jobs=1)(delayed(_coord_batch_n)(lo, hi, train_data, prefix = 'item') for lo, hi in zip(start_idx, end_idx))
t2 = time.time()
print ('Time : %d seconds'%(t2-t1))
pass


def _load_coord_matrix(start_idx, end_idx, nrow, ncol, prefix = 'item'):
    X = sparse.csr_matrix((nrow, ncol), dtype='float32')

    for lo, hi in zip(start_idx, end_idx):
        coords = np.load(os.path.join(DATA_DIR, 'negative-co-temp', 'negative_%s_coo_%d_%d.npy' % (prefix, lo, hi)))

        rows = coords[:, 0]
        cols = coords[:, 1]

        tmp = sparse.coo_matrix((np.ones_like(rows), (rows, cols)), shape=(nrow, ncol), dtype='float32').tocsr()
        X = X + tmp

        print("%s %d to %d finished" % (prefix, lo, hi))
        sys.stdout.flush()
    return X


X, Y = None, None
print ('Loading item item negative_co-occurrence matrix and saving to pickle file for fast loading')
t1 = time.time()
start_idx = range(0, n_users, batch_size)
list_start_idx = list(start_idx)
end_idx = list_start_idx[1:] + [n_users]
X = _load_coord_matrix(start_idx, end_idx, n_items, n_items, prefix = 'item') #item item co-occurrence matrix
print ('dumping matrix ...')
save_pickle(X, os.path.join(DATA_DIR,'negative_item_item_cooc.dat'))
t2 = time.time()
print ('Time : %d seconds'%(t2-t1))

Generating item item negative_co-occurrence matrix
Time : 6 seconds
Loading item item negative_co-occurrence matrix and saving to pickle file for fast loading
item 0 to 5000 finished
item 5000 to 10000 finished
item 10000 to 15000 finished
item 15000 to 20000 finished
item 20000 to 20279 finished
dumping matrix ...
Saved drive/MyDrive/dl2022/negative_item_item_cooc.dat..
Time : 0 seconds


In [94]:
parser = argparse.ArgumentParser("Description: Running multi-embedding recommendation - RME model")
parser.add_argument('-f')
parser.add_argument('--data_path', default='data', type=str, help='path to the data')
parser.add_argument('--saved_model_path', default='MODELS', type=str, help='path to save the optimal learned parameters')
parser.add_argument('--s', default=1, type=int, help='a pre-defined shifted value for measuring SPPMI')
parser.add_argument('--model', default='rme', type=str, help='the model to run: rme, cofactor')
parser.add_argument('--n_factors', default=40, type=int, help='number of hidden factors for user/item representation')
parser.add_argument('--reg', default=1.0, type=float, help='regularization for user and item latent factors (alpha, beta)')
parser.add_argument('--reg_embed', default=1.0, type=float, help='regularization for user and item context latent factors (gamma, delta, theta)')
parser.add_argument('--dataset', default="ml10m", type=str, help='dataset')
parser.add_argument('--neg_item_inference', default=0, type=int, help='if there is no available disliked items, set this to 1 to infer '
                                                                      'negative items for users using our user-oriented EM like algorithm')
parser.add_argument('--neg_sample_ratio', default=0.2, type=float, help='negative sample ratio per user. If a user consumed 10 items, and this'
                                                                        'neg_sample_ratio = 0.2 --> randomly sample 2 negative items for the user')

args = parser.parse_args()

In [85]:
DATA_DIR =  os.path.join('drive', 'MyDrive', 'dl2022')
gc.DATA_DIR = DATA_DIR
gc.SAVED_MODLE_DIR = args.saved_model_path
gc.PRED_DIR = os.path.join(DATA_DIR, 'prediction-temp')
SHIFTED_K_VALUE = args.s
NEGATIVE_SAMPLE_RATIO = args.neg_sample_ratio
save_dir = os.path.join(DATA_DIR, 'model_tmp_res')
n_components = args.n_factors
lam = args.reg
lam_emb = args.reg_embed

unique_uid = list()
with open(os.path.join(DATA_DIR, 'unique_uid.txt'), 'r') as f:
    for line in f:
        unique_uid.append(line.strip())

unique_movieId = list()
with open(os.path.join(DATA_DIR, 'unique_sid.txt'), 'r') as f:
    for line in f:
        unique_movieId.append(line.strip())
n_items = len(unique_movieId)
n_users = len(unique_uid)
n_items = len(unique_movieId)
print(n_users, n_items)

def load_data(csv_file, shape=(n_users, n_items)):
    tp = pd.read_csv(csv_file)
    rows, cols = np.array(tp['userId']), np.array(tp['movieId'])
    seq = np.concatenate((rows[:, None], cols[:, None], np.ones((rows.size, 1), dtype='int')), axis=1)
    data = sparse.csr_matrix((np.ones_like(rows), (rows, cols)), dtype=np.int16, shape=shape)
    return data, seq, tp

def get_row(M, i):
    # get the row i of sparse matrix:
    lo, hi = M.indptr[i], M.indptr[i + 1]
    return lo, hi, M.data[lo:hi], M.indices[lo:hi]


def convert_to_SPPMI_matrix(M, max_row, shifted_K=1):
    # if we sum the co-occurrence matrix by row wise or column wise --> we have an array that contain the #(i) values
    obj_counts = np.asarray(M.sum(axis=1)).ravel()
    total_obj_pairs = M.data.sum()
    M_sppmi = M.copy()
    for i in xrange(max_row):
        lo, hi, data, indices = get_row(M, i)
        M_sppmi.data[lo:hi] = np.log(data * total_obj_pairs / (obj_counts[i] * obj_counts[indices]))
    M_sppmi.data[M_sppmi.data < 0] = 0
    M_sppmi.eliminate_zeros()
    if shifted_K == 1:
        return M_sppmi
    else:
        M_sppmi.data -= np.log(shifted_K)
        M_sppmi.data[M_sppmi.data < 0] = 0
        M_sppmi.eliminate_zeros()
    return M_sppmi

20279 6051


In [96]:
#initialize with WMF:
import wmf
import pyll
from scipy import sparse
import glob
import os

import produce_negative_embedding as pne # produce_negative_embedding 이 패키지가 없음...

ModuleNotFoundError: ignored

In [97]:
!pip install produce_negative_embedding

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement produce_negative_embedding (from versions: none)
ERROR: No matching distribution found for produce_negative_embedding


In [99]:
if args.neg_item_inference:
    #initialize with WMF:
    import wmf
    import rec_eval
    from scipy import sparse
    import produce_negative_embedding as pne
    import glob
    import os

    def softmax(x):
        """Compute softmax values for each ranked list."""
        # We want the item with higher ranking score have lower prob to be withdrawn as negative instances#
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum(axis=0)
    def compute_neg_prob(ranks):
        return softmax(np.negative(ranks))
    def _make_prediction(train_data, Et, Eb, user_idx, batch_users, mu=None,
                         vad_data=None):
        n_songs = train_data.shape[1]
        # exclude examples from training and validation (if any)
        item_idx = np.zeros((batch_users, n_songs), dtype=bool)
        item_idx[train_data[user_idx].nonzero()] = True
        if vad_data is not None:
            item_idx[vad_data[user_idx].nonzero()] = True
        X_pred = Et[user_idx].dot(Eb)
        if mu is not None:
            if isinstance(mu, np.ndarray):
                assert mu.size == n_songs  # mu_i
                X_pred *= mu
            elif isinstance(mu, dict):  # func(mu_ui)
                params, func = mu['params'], mu['func']
                args = [params[0][user_idx], params[1]]
                if len(params) > 2:  # for bias term in document or length-scale
                    args += [params[2][user_idx]]
                if not callable(func):
                    raise TypeError("expecting a callable function")
                X_pred *= func(*args)
            else:
                raise ValueError("unsupported mu type")
        X_pred[item_idx] = np.inf
        return X_pred

    def gen_neg_instances(train_data, U, VT, user_idx, neg_ratio = 1.0, iter = 0):
        print ('Job start... %d to %d'%(user_idx.start, user_idx.stop))
        #if user_idx.start != 99000: return
        batch_users = user_idx.stop - user_idx.start
        X_pred = _make_prediction(train_data, U, VT, user_idx, batch_users, vad_data=vad_data)

        rows = []
        cols = []
        total_lost = 0
        for idx, uid in enumerate(range(user_idx.start, user_idx.stop)):
            num_pos = train_data[uid].count_nonzero()
            num_neg = int(num_pos * neg_ratio)
            if num_neg <= 0: continue
            ranks = X_pred[idx]
            neg_withdrawn_prob = compute_neg_prob(ranks)
            # print (neg_withdrawn_prob)
            neg_instances = list(set(np.random.choice(range(n_items), num_neg, p = neg_withdrawn_prob)))
            #rows = rows + len(neg_instances)*[uid]
            #uid_dup = np.empty(len(neg_instances))
            #uid_dup.fill(uid)
            if uid < 0: print ('error with %d to %d'%(user_idx.start, user_idx.stop))
            #rows = rows + uid_dup
            rows = np.append(rows, np.full( len(neg_instances), uid )  )
            cols = np.append(cols, neg_instances)
        # print 'check for neg values: ', np.sum(rows <0)
        # print 'check for neg values: ', np.sum(cols <0)
        if  len(rows) > 0:
            path = os.path.join(DATA_DIR, 'sub_dataframe_iter_%d_idxstart_%d.csv' % (iter, user_idx.start))
            assert len(rows) == len(cols)
            with open(path, 'w') as writer:
                for i in range(len(rows)): writer.write(str(rows[i]) + "," + str(cols[i]) + '\n')
                writer.flush()
            #df = pd.DataFrame({'uid':rows, 'sid':cols}, columns=["uid", "sid"], dtype=np.int16)
            #df.to_csv(path, sep=",",header=False, index = False)
        # return df
    U, V = None, None



    vad_data, vad_raw, vad_df = load_data(os.path.join(DATA_DIR, 'validation.csv'))
    train_data, train_raw, train_df = load_data(os.path.join(DATA_DIR, 'train.csv'))
    test_data, test_raw, test_df = load_data(os.path.join(DATA_DIR, 'test.csv'))
    U, V = wmf.decompose(train_data, vad_data, num_factors= n_components)
    VT = V.T
    iter, max_iter = 0, 10

    #load postivie information
    X = load_pickle(os.path.join(DATA_DIR, 'item_item_cooc.dat'))
    Y = load_pickle(os.path.join(DATA_DIR, 'user_user_cooc.dat'))
    X_sppmi = convert_to_SPPMI_matrix(X, max_row=n_items, shifted_K=SHIFTED_K_VALUE)
    Y_sppmi = convert_to_SPPMI_matrix(Y, max_row=n_users, shifted_K=SHIFTED_K_VALUE)

    best_ndcg100 = 0.0
    best_iter = 1
    early_stopping = False
    while (iter < max_iter and not early_stopping):
        ################ Expectation step: ######################
        user_slices = rec_eval.user_idx_generator(n_users, batch_users=5000)
        print ('GENERATING NEGATIVE INSTANCES ...')
        t1 = time.time()
        df = Parallel(n_jobs=16)(delayed(gen_neg_instances)(train_data, U, VT, user_idx, neg_ratio = NEGATIVE_SAMPLE_RATIO, iter = iter)
                                      for user_idx in user_slices)
        t2 = time.time()
        print ('Time : %d seconds' % (t2 - t1))


        print ('merging to one file ...')
        t1 = time.time()
        neg_file_out = os.path.join(DATA_DIR, 'train_neg_iter_%d.csv' % (iter))
        with open(neg_file_out, 'w') as writer:
            writer.write('userId,movieId\n')
        # os.system("echo uid,sid >> " + neg_file_out)
        for f in glob.glob(os.path.join(DATA_DIR, 'sub_dataframe_iter*')):
            os.system("cat " + f + " >> " + neg_file_out)
                # with open(f, 'rb') as reader:
                #
                #     writer.write(reader.readline())
            # writer.flush()
        #clean
        for f in glob.glob(os.path.join(DATA_DIR, 'sub_dataframe_iter*')):
            os.remove(f)

        t2 = time.time()
        print ('Time : %d seconds' % (t2 - t1))
        # neg_train_df = pd.concat(df)
        # neg_train_df.to_csv(neg_file_out, index = False)
        #########################################################

        ################ maximization step:######################
        print ('GENERATING NEGATIVE EMBEDDINGS ...')
        t1 = time.time()
        train_neg_data, _, train_neg_df = load_data(neg_file_out, shape=(n_users, n_items))
        #build the negative info:
        X_neg, _ = pne.produce_neg_embeddings(DATA_DIR, train_neg_data, n_users, n_items, iter = iter)
        X_neg_sppmi = convert_to_SPPMI_matrix(X_neg, max_row=n_items, shifted_K=SHIFTED_K_VALUE)
        Y_neg_sppmi = None
        t2 = time.time()
        print ('Time : %d seconds' % (t2 - t1))

        # build the model
        print ('build the model...')
        t1 = time.time()
        runner = ModelRunner(train_data, vad_data, None, X_sppmi, X_neg_sppmi, Y_sppmi, None, save_dir=save_dir)
        U, V, ndcg100 = runner.run("rme", n_jobs = 1,
                                         lam=lam, lam_emb=lam_emb, n_components = n_components, ret_params_only = 1)
        t2 = time.time()
        print ('Time : %d seconds' % (t2 - t1))
        print ('*************************************ITER %d ******************************************' % iter)
        print ('NDCG@100 at this iter:',ndcg100)
        #
        if best_ndcg100 < ndcg100:
            best_iter = iter
            best_ndcg100 = ndcg100
        else:
            early_stopping = True
        iter += 1
        #########################################################
    print ('Max NDCG@100: %f , at iter: %d'%(best_ndcg100, best_iter))
    best_train_neg_file = os.path.join(DATA_DIR, 'train_neg_iter_%d.csv' % (best_iter))
    best_train_neg_file_newname = os.path.join(DATA_DIR, 'train_neg.csv')
    best_train_emb_file = os.path.join(DATA_DIR, 'negative_item_item_cooc_iter%d.dat' % (best_iter))
    best_train_emb_file_newname = os.path.join(DATA_DIR, 'negative_item_item_cooc.dat')
    print ('renaming from %s to %s'%(best_train_neg_file, best_train_neg_file_newname))
    os.rename(best_train_neg_file, best_train_neg_file_newname)
    print ('renaming from %s to %s' % (best_train_emb_file, best_train_emb_file_newname))
    os.rename(best_train_emb_file, best_train_emb_file_newname)
    #cleaning
    for i in range(max_iter):
        if i == best_iter: continue
        if early_stopping and (i > best_iter + 1): break
        del_file = os.path.join(DATA_DIR, 'train_neg_iter_%d.csv' % ( i))
        os.remove(del_file)
        del_file = os.path.join(DATA_DIR, 'negative_item_item_cooc_iter%d.dat' % (i))
        os.remove(del_file)




LOAD_NEGATIVE_MATRIX = True
if args.model.lower() != 'rme':
    LOAD_NEGATIVE_MATRIX = False
recalls = np.zeros(5, dtype=np.float32) #store results of topk recommendation in range [5, 10, 20, 50, 100]
ndcgs = np.zeros(5, dtype=np.float32)
maps = np.zeros(5, dtype=np.float32)
print ('*************************************lam =  %.3f ******************************************' % lam)
print ('*************************************lam embedding =  %.3f ******************************************' % lam_emb)

# train_data, train_raw, train_df = load_data(os.path.join(DATA_DIR, 'train_fold%d.csv'%FOLD))
vad_data, vad_raw, vad_df = load_data(os.path.join(DATA_DIR, 'validation.csv'))
test_data, test_raw, test_df = load_data(os.path.join(DATA_DIR, 'test.csv'))
train_data, train_raw, train_df = load_data(os.path.join(DATA_DIR, 'train.csv' ))

print ('loading pro_pro_cooc.dat')
t1 = time.time()
X = load_pickle(os.path.join(DATA_DIR, 'item_item_cooc.dat'))
t2 = time.time()
print ('[INFO]: sparse matrix size of item item co-occurrence matrix: %d mb\n' % (
    (X.data.nbytes + X.indices.nbytes + X.indptr.nbytes) / (1024 * 1024)))
print ('Time : %d seconds' % (t2 - t1))

print ('loading user_user_cooc.dat')
t1 = time.time()
Y = load_pickle(os.path.join(DATA_DIR, 'user_user_cooc.dat'))
t2 = time.time()
print ('[INFO]: sparse matrix size of user user co-occurrence matrix: %d mb\n' % (
    (Y.data.nbytes + Y.indices.nbytes + Y.indptr.nbytes) / (1024 * 1024)))
print ('Time : %d seconds' % (t2 - t1))
################# LOADING NEGATIVE CO-OCCURRENCE MATRIX ########################################

if LOAD_NEGATIVE_MATRIX:
    print ('test loading negative_pro_pro_cooc.dat')
    t1 = time.time()
    X_neg = load_pickle(os.path.join(DATA_DIR, 'negative_item_item_cooc.dat'))
    t2 = time.time()
    print ('[INFO]: sparse matrix size of negative item item co-occurrence matrix: %d mb\n' % (
        (X_neg.data.nbytes + X_neg.indices.nbytes + X_neg.indptr.nbytes) / (1024 * 1024)))
    print ('Time : %d seconds' % (t2 - t1))


################################################################################################
########## converting CO-OCCURRENCE MATRIX INTO Shifted Positive Pointwise Mutual Information (SPPMI) matrix ###########
####### We already know the user-user co-occurrence matrix Y and item-item co-occurrence matrix X

print ('converting co-occurrence matrix into sppmi matrix')
t1 = time.time()
X_sppmi = convert_to_SPPMI_matrix(X, max_row=n_items, shifted_K=SHIFTED_K_VALUE)
Y_sppmi = convert_to_SPPMI_matrix(Y, max_row=n_users, shifted_K=SHIFTED_K_VALUE)
t2 = time.time()
print ('Time : %d seconds' % (t2 - t1))
# if DEBUG_MODE:
#     print 'item sppmi matrix'
#     print X_sppmi
#     print 'user sppmi matrix'
#     print Y_sppmi

############### Negative SPPMI matrix ##########################
X_neg_sppmi = None
Y_neg_sppmi = None
if LOAD_NEGATIVE_MATRIX:
    print ('converting negative co-occurrence matrix into sppmi matrix')
    t1 = time.time()
    X_neg_sppmi = convert_to_SPPMI_matrix(X_neg, max_row=n_items, shifted_K=SHIFTED_K_VALUE)
    t2 = time.time()
    print ('Time : %d seconds' % (t2 - t1))
################################################################


######## Finally, we have train_data, vad_data, test_data,
# X_sppmi: item item Shifted Positive Pointwise Mutual Information matrix
# Y_sppmi: user-user       Shifted Positive Pointwise Mutual Information matrix


print ('Training data', train_data.shape)
print ('Validation data', vad_data.shape)
print ('Testing data', test_data.shape)

n_jobs = 1  # default value
model_type = 'model2'  # default value
if os.path.exists(save_dir):
    #clearning folder
    lst = glob.glob(os.path.join(save_dir, '*.*'))
    for f in lst:
        os.remove(f)
else:
    os.mkdir(save_dir)


runner = ModelRunner(train_data, vad_data, test_data, X_sppmi, X_neg_sppmi, Y_sppmi, Y_neg_sppmi,
                       save_dir=save_dir)

start = time.time()
if args.model == 'wmf':
    (recalls, ndcgs, maps) = runner.run("wmf", n_jobs=n_jobs, lam=lam,
                                                         saved_model = True,
                                                         n_components = n_components)
if args.model == 'cofactor':
    (recalls, ndcgs, maps) = runner.run("cofactor", n_jobs=n_jobs,
                                                        lam=lam,
                                                         saved_model=True,
                                                         n_components=n_components)
if args.model == 'rme':
    (recalls, ndcgs, maps) = runner.run("rme", n_jobs=n_jobs,lam=lam, lam_emb = lam_emb,
                                                         saved_model=True,
                                                         n_components=n_components)
end = time.time()
print ('total running time: %d seconds'%(end-start))
for idx, topk in enumerate([5, 10, 20, 50, 100]):
    print ('top-%d results: recall@%d = %.4f, ndcg@%d = %.4f, map@%d = %.4f'%(topk,
                                                                                  topk, recalls[idx],
                                                                                  topk, ndcgs[idx],
                                                                                  topk, maps[idx]))

*************************************lam =  1.000 ******************************************
*************************************lam embedding =  1.000 ******************************************
loading pro_pro_cooc.dat


AttributeError: ignored

In [ ]:
# load_pickle 했을 때 왜 none값을 갖고 오는지... 

In [101]:
# 논문에서는 아래와 같은 결과를 엉었음.

"""
top-5 results: recall@5 = 0.1559, ndcg@5 = 0.1613, map@5 = 0.1076
top-10 results: recall@10 = 0.1513, ndcg@10 = 0.1547, map@10 = 0.0851
top-20 results: recall@20 = 0.1477, ndcg@20 = 0.1473, map@20 = 0.0669
top-50 results: recall@50 = 0.1819, ndcg@50 = 0.1553, map@50 = 0.0562
top-100 results: recall@100 = 0.2533, ndcg@100 = 0.1825, map@100 = 0.0579
"""

'\ntop-5 results: recall@5 = 0.1559, ndcg@5 = 0.1613, map@5 = 0.1076\ntop-10 results: recall@10 = 0.1513, ndcg@10 = 0.1547, map@10 = 0.0851\ntop-20 results: recall@20 = 0.1477, ndcg@20 = 0.1473, map@20 = 0.0669\ntop-50 results: recall@50 = 0.1819, ndcg@50 = 0.1553, map@50 = 0.0562\ntop-100 results: recall@100 = 0.2533, ndcg@100 = 0.1825, map@100 = 0.0579\n'